# PyEhsa Demo - São Paulo Grid Analysis


In [59]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(''), '..', 'src'))

import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from datetime import datetime, timedelta

from pyehsa.emerging_hotspot_analysis import EmergingHotspotAnalysis
from pyehsa.ehsa_plotting import EhsaPlotting

np.random.seed(42)
print("Libraries imported successfully")


Libraries imported successfully


In [60]:
# Create synthetic data - 15x15 grid covering São Paulo city
from shapely.geometry import Polygon

# São Paulo city bounds (approximate)
center_lat, center_lon = -23.5505, -46.6333
grid_size = 15  # 15x15 = 225 areas
step = 0.015   # Grid cell size (~1.7km x 1.7km)

data = []
for i in range(grid_size):
    for j in range(grid_size):
        # Center of grid cell  
        lat = center_lat + (i - grid_size//2) * step
        lon = center_lon + (j - grid_size//2) * step
        location_id = f'SP_{i:02d}_{j:02d}'
        
        # Create polygon for grid cell
        half_step = step / 2
        polygon = Polygon([
            (lon - half_step, lat - half_step),
            (lon + half_step, lat - half_step), 
            (lon + half_step, lat + half_step),
            (lon - half_step, lat + half_step),
            (lon - half_step, lat - half_step)
        ])
        
        for month in range(8):  # 8 months for more temporal data
            time_period = datetime(2024, 1, 1) + timedelta(days=30*month)
            
            # Base value with spatial variation
            base_value = np.random.poisson(8)
            
            # Create multiple hotspot patterns
            # Hotspot 1: Southeast area (emerging)
            if i >= 10 and j >= 10 and month >= 3:
                base_value += (month - 2) * 6
                
            # Hotspot 2: Northwest area (persistent)  
            if i <= 4 and j <= 4:
                base_value += 5 + np.random.poisson(3)
                
            # Hotspot 3: Central stripe (oscillating)
            if 6 <= i <= 8 and month % 2 == 0:
                base_value += 8
            
            # Add noise
            value = base_value + np.random.normal(0, 1.5)
            value = max(0, value)
            
            data.append({
                'location_id': location_id,
                'time_period': time_period,
                'value': value,
                'geometry': polygon
            })

gdf = gpd.GeoDataFrame(data, geometry='geometry', crs='EPSG:4326')
print(f"Dataset: {len(gdf)} observations, {gdf['location_id'].nunique()} grid cells")


Dataset: 1800 observations, 225 grid cells


In [61]:
print(gdf.dtypes)
gdf

location_id            object
time_period    datetime64[ns]
value                 float64
geometry             geometry
dtype: object


,location_id,time_period,value,geometry
0,SP_00_00,2024-01-01,13.128683,"POLYGON ((-46.7458 -23.663, -46.7308 -23.663, ..."
1,SP_00_00,2024-01-31,12.212245,"POLYGON ((-46.7458 -23.663, -46.7308 -23.663, ..."
2,SP_00_00,2024-03-01,13.099042,"POLYGON ((-46.7458 -23.663, -46.7308 -23.663, ..."
3,SP_00_00,2024-03-31,14.562459,"POLYGON ((-46.7458 -23.663, -46.7308 -23.663, ..."
4,SP_00_00,2024-04-30,14.309042,"POLYGON ((-46.7458 -23.663, -46.7308 -23.663, ..."
...,...,...,...,...
1795,SP_14_14,2024-03-31,11.792031,"POLYGON ((-46.5358 -23.453, -46.5208 -23.453, ..."
1796,SP_14_14,2024-04-30,20.311649,"POLYGON ((-46.5358 -23.453, -46.5208 -23.453, ..."
1797,SP_14_14,2024-05-30,24.728419,"POLYGON ((-46.5358 -23.453, -46.5208 -23.453, ..."
1798,SP_14_14,2024-06-29,32.578209,"POLYGON ((-46.5358 -23.453, -46.5208 -23.453, ..."


In [62]:
# Run EHSA analysis
results = EmergingHotspotAnalysis.emerging_hotspot_analysis(
    gdf,
    region_id_field='location_id',
    time_period_field='time_period', 
    value='value',
    k=1,
    nsim=99
)


2025-10-13 10:52:46 - INFO - 🚀 Starting Emerging Hotspot Analysis
2025-10-13 10:52:46 - INFO - 📊 Input DataFrame shape: (1800, 4)
2025-10-13 10:52:46 - INFO - 🎯 Analysis parameters:
2025-10-13 10:52:46 - INFO -    - Region ID field: location_id
2025-10-13 10:52:46 - INFO -    - Time period field: time_period
2025-10-13 10:52:46 - INFO -    - Value field: value
2025-10-13 10:52:46 - INFO -    - Random seed: 77
2025-10-13 10:52:46 - INFO -    - Time lags (k): 1
2025-10-13 10:52:46 - INFO -    - Simulations (nsim): 99
2025-10-13 10:52:46 - INFO - 📈 Data overview:
2025-10-13 10:52:46 - INFO -    - Total rows: 1,800
2025-10-13 10:52:46 - INFO -    - Unique regions: 225
2025-10-13 10:52:46 - INFO -    - Unique time periods: 8
2025-10-13 10:52:46 - INFO -    - Value range: [0.000, 43.409]
2025-10-13 10:52:46 - INFO -    - Missing values in value: 0
2025-10-13 10:52:46 - INFO - 🔧 Step 1: Validating and cleaning input data...
2025-10-13 10:52:46 - INFO - ✅ Data validation completed
2025-10-13 1

Geometries are already Shapely objects, creating GeoDataFrame...
Setting CRS...
Creating complete spacetime cube:
  - 225 locations
  - 8 time periods
  - 1800 total combinations
  - Original data: 1800 rows
  - Complete cube: 1800 rows
  - Missing combinations filled with NAs: 0
Spacetime cube dimensions:
  - 225 locations
  - 8 time periods
  - 1800 total observations


2025-10-13 10:52:59 - INFO - ✅ Gi* statistics calculated in 12.86s using spacetime method
2025-10-13 10:52:59 - INFO -    - Output shape: (1800, 7)
2025-10-13 10:52:59 - INFO -    - Used 99 simulations for p-values
2025-10-13 10:52:59 - INFO - 🎯 Step 5: Performing emerging hotspot classification...
2025-10-13 10:52:59 - INFO - ✅ EHSA classification completed in 0.09s
2025-10-13 10:52:59 - INFO -    - Results shape: (225, 8)
2025-10-13 10:52:59 - INFO - ==================================================
2025-10-13 10:52:59 - INFO - 📋 ANALYSIS RESULTS SUMMARY
2025-10-13 10:52:59 - INFO - ==================================================
2025-10-13 10:52:59 - INFO - ⏱️  Total execution time: 0:00:12.995576
2025-10-13 10:52:59 - INFO - 🏷️  Classification distribution:
2025-10-13 10:52:59 - INFO -    - no pattern detected: 115 regions (51.1%)
2025-10-13 10:52:59 - INFO -    - sporadic coldspot: 57 regions (25.3%)
2025-10-13 10:52:59 - INFO -    - oscilating hotspot: 17 regions (7.6%)
2025-

Gi* range: [-3.813, 10.512]
Significant observations (p<=0.01): 272/1800
Significant observations (p<=0.05): 657/1800
DEBUG - Sample of borderline cases:
  Obs 0: Gi*=0.926, p=0.2400, sig=False
  Obs 10: Gi*=-1.659, p=0.0600, sig=False
  Obs 20: Gi*=0.974, p=0.2500, sig=False
  Obs 30: Gi*=1.147, p=0.2300, sig=False
  Obs 40: Gi*=-1.302, p=0.1000, sig=False
DEBUG - Neighbor permutation test:
  Original neighbors: [[15, 0], [17, 2, 16, 1], [1, 18, 16, 17, 3, 2]]
  Permuted neighbors: [[6, 9], [6, 5, 7, 0], [8, 0, 5, 6, 7, 3]]
  Neighbor structure changed: True
Total regions to process: 225
   No pattern #1: SP_00_00, Gi* range [0.75, 2.38], 0/10 significant
   No pattern #2: SP_00_01, Gi* range [0.19, 5.23], 0/10 significant
✓ SP_00_02: sporadic hotspot
✓ SP_00_03: sporadic hotspot
   No pattern #3: SP_00_04, Gi* range [0.29, 4.46], 0/10 significant
✓ SP_00_07: consecutive coldspot
✓ SP_00_08: sporadic coldspot
✓ SP_00_09: sporadic coldspot
✓ SP_00_12: sporadic coldspot
✓ SP_00_13: new 

In [63]:
# Show results
print("Patterns identified:")
print(results['classification'].value_counts())
print("\nFirst results:")
results.head()


Patterns identified:
classification
no pattern detected     115
sporadic coldspot        57
oscilating hotspot       17
consecutive hotspot      13
sporadic hotspot         11
consecutive coldspot      7
new coldspot              3
new hotspot               2
Name: count, dtype: int64

First results:


,location_id,classification,classification_details,mann_kendall_details,spatial_context_summary,location_data,tau,p_value
0,SP_00_00,no pattern detected,{'reason': 'Conditions for specific hotspot/co...,"{'inputs': {'time_periods_count': 8, 'gi_star_...","{'neighbors_config': ['SP_01_00', 'SP_00_00'],...","[{'time_period': 2024-01-01 00:00:00, 'value':...",-0.214286,0.536187
1,SP_00_01,no pattern detected,{'reason': 'Conditions for specific hotspot/co...,"{'inputs': {'time_periods_count': 8, 'gi_star_...","{'neighbors_config': ['SP_01_02', 'SP_00_02', ...","[{'time_period': 2024-01-01 00:00:00, 'value':...",-0.500000,0.107762
2,SP_00_02,sporadic hotspot,{'criteria_met': ['Proportion of significant h...,"{'inputs': {'time_periods_count': 8, 'gi_star_...","{'neighbors_config': ['SP_00_01', 'SP_01_03', ...","[{'time_period': 2024-01-01 00:00:00, 'value':...",-0.428571,0.173546
3,SP_00_03,sporadic hotspot,{'criteria_met': ['Proportion of significant h...,"{'inputs': {'time_periods_count': 8, 'gi_star_...","{'neighbors_config': ['SP_00_04', 'SP_01_03', ...","[{'time_period': 2024-01-01 00:00:00, 'value':...",-0.500000,0.107762
4,SP_00_04,no pattern detected,{'reason': 'Conditions for specific hotspot/co...,"{'inputs': {'time_periods_count': 8, 'gi_star_...","{'neighbors_config': ['SP_00_05', 'SP_01_03', ...","[{'time_period': 2024-01-01 00:00:00, 'value':...",-0.714286,0.018741


In [64]:
# Merge geometry with results  
locations = gdf[['location_id', 'geometry']].drop_duplicates()
ehsa_df = results.merge(locations, left_on=results.columns[0], right_on='location_id')

# Create EHSA map
ehsa_map = EhsaPlotting.plot_ehsa_map_interactive(
    df=ehsa_df,
    region_id_field="location_id",
    title="São Paulo Emerging Hotspots"
)
ehsa_map.show()


/Users/lucasazevedo/Desktop/git/tyrell-ehsa/PyEhsa/src/pyehsa/ehsa_plotting.py:189: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/Users/lucasazevedo/Desktop/git/tyrell-ehsa/PyEhsa/src/pyehsa/ehsa_plotting.py:190: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/Users/lucasazevedo/Desktop/git/tyrell-ehsa/PyEhsa/src/pyehsa/ehsa_plotting.py:208: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


/Users/lucasazevedo/Desktop/git/tyrell-ehsa/PyEhsa/src/pyehsa/ehsa_plotting.py:209: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSe

In [67]:
EhsaPlotting.open_visualization_tool()

Opening EHSA Visualization Tool in your default browser...
File location: /var/folders/vw/lchjhtcn06jgcb946y77vpb80000gn/T/ehsa_viz_qs2j_ioy/ehsa_visualization.html


'/var/folders/vw/lchjhtcn06jgcb946y77vpb80000gn/T/ehsa_viz_qs2j_ioy/ehsa_visualization.html'

In [66]:
ehsa_df.to_csv('ehsa_df.csv', index=False)